In [7]:
days = 20*5

DemandA3 = [3000 2000 2000 2820 2900 0]

Dfactor = 60;
Efactor = 240;
A31factor = 20;

InvA3 = zeros(days+50,1)
InvD = zeros(days+50,1)
InvE = zeros(days+50,1)
InvA31 = zeros(days+50,1)

InvA31[1:end] = 10;

cycleA3 = 643

t=1;
demandMonth = 1
nOrders=0
noOrder = true
while t < days
    if t >= (demandMonth*20)
        InvA3[t:end] -= DemandA3[demandMonth]
        demandMonth += 1
    end
    if InvA3[t] > DemandA3[demandMonth]
        keepProduction = false
    else
        keepProduction = true
    end
    Dleft = (DemandA3[demandMonth] - InvA3[t]) * Dfactor
    Eleft = (DemandA3[demandMonth] - InvA3[t]) * Efactor
    A31left = (DemandA3[demandMonth] - InvA3[t]) * A31factor
    if ((InvD[t] < Dleft) || (InvE[t] < Eleft) || (InvA31[t] < A31left)) && noOrder && keepProduction
        nOrders+=1
        tLast = t
        tAdd = round(Int64,2*randn(1)[1] + 5)
        if tAdd < 0
            t+=0
        else
            t+=tAdd
        end
        
        InvD[tLast:t] = InvD[tLast]
        InvE[tLast:t] = InvE[tLast]
        InvA31[tLast:t] = InvA31[tLast]
        InvA3[tLast:t] = InvA3[tLast]
        
        InvD[t:end] += max(DemandA3[demandMonth]* Dfactor, cycleA3 * Dfactor * 7)
        InvE[t:end] += max(DemandA3[demandMonth]* Efactor, cycleA3 * Efactor * 7)
        InvA31[t:end] += max(DemandA3[demandMonth]* A31factor, cycleA3 * A31factor * 7)
    else
        if InvA3[t] > DemandA3[demandMonth]
            keepProduction = false
        else
            keepProduction = true
        end
        while ((InvD[t]-Dfactor)>0) && ((InvE[t]-Efactor)>0) && ((InvA31[t]-A31factor)>0) && ((t+1)<= days) && keepProduction          
            
            prod = round(Int64, 200*randn(1)[1] + cycleA3/2)
            nD = div(InvD[t], Dfactor)
            nE = div(InvE[t], Efactor)
            nA31 = div(InvA31[t], A31factor)
            
            nA3 = min(nD, nE, nA31, prod)
            
            InvA3[t:end] += nA3
            InvD[t:end] -= nA3*Dfactor
            InvE[t:end] -= nA3*Efactor
            InvA31[t:end] -= nA3*A31factor            
            defect = rand()/10
            
            InvA3[t:end] -= round(Int64, InvA3[t+1]*defect)
            if InvA3[t] > DemandA3[demandMonth]
                break
            end
            t+=1
            if t >= (demandMonth*20)
                InvA3[t:end] -= DemandA3[demandMonth]
                demandMonth += 1
            end
            Dleft = (DemandA3[demandMonth] - InvA3[t]) * Dfactor
            Eleft = (DemandA3[demandMonth] - InvA3[t]) * Efactor
            A31left = (DemandA3[demandMonth] - InvA3[t]) * A31factor
            if ((InvD[t] < Dleft) || (InvE[t] < Eleft) || (InvA31[t] < A31left)) && noOrder
                noOrder = false
                nOrders += 1
                tLast = t
                tAdd = round(Int64,2*randn(1)[1] + 5)
                if tAdd < 0
                    tOrder=t
                else
                    tOrder=tAdd+t
                end

                InvD[tLast:tOrder] = InvD[tLast]
                InvE[tLast:tOrder] = InvE[tLast]
                InvA31[tLast:tOrder] = InvA31[tLast]
                InvA3[tLast:tOrder] = InvA3[tLast]

                #InvD[tOrder:end] += max(Dleft* Dfactor, cycleA3 * Dfactor * 7)
                #InvE[tOrder:end] += max(Eleft* Efactor, cycleA3 * Efactor * 7)
                #InvA31[tOrder:end] += max(A31left* A31factor, cycleA3 * A31factor * 7)
            end
            if !noOrder
                if !(((InvD[t]-Dfactor)>0) && ((InvE[t]-Efactor)>0) && ((InvA31[t]-A31factor)>0) && ((t+1)<= days) && keepProduction)
                    noOrder = true
                    t=tOrder
                    InvD[tOrder:end] += max(Dleft* Dfactor, cycleA3 * Dfactor * 7)
                    InvE[tOrder:end] += max(Eleft* Efactor, cycleA3 * Efactor * 7)
                    InvA31[tOrder:end] += max(A31left* A31factor, cycleA3 * A31factor * 7)
                end
            end
            
        end
    end
    t+=1
end




In [8]:
using Plots
plot(1:(days+50), InvD/Dfactor, label="D scaled", title="Inventory over 5 months")
plot!(1:(days+50), InvE/Efactor, label = "E scaled")
plot!(1:(days+50), InvA31/A31factor, label="A31 scaled")
plot!(1:(days+50), InvA3, label="A3 actual")
using Images
using FileIO
savefig("a3invall.png")

In [9]:
plot(1:(days+50), InvA3, label="A3", title="Inventory of A3")
using Images
using FileIO
savefig("a3inv.png")